# MNIST : RdN - MLP

Plusieur étapes à suivre pour entrainer un modèle : 
1. Récupérer les données (divisées en deux ensembles : entraînement + test)
2. Les préparer pour qu'elles soient adaptées au modèle
3. Construction du modèle (Les couches + compilation)
4. Entraînement du modèle
5. Vérification du modèle

#### Étape 1 : récupérer les données

In [2]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#### Étape 2 : préparer les données

In [3]:
# On convertit les 6000 images en Scalaires (1D) 
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

# On code de manière catégorielle les labels. 
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

**One-hot encoding** : Dernière layer : activation='softmax' => elle s'attend donc à un input du type [0, 0, 0, 0, 1, 0, 0, 0, 0, 0] = 4. D'où la convertion des labels

#### Étape 3 : Construction du modèle MLP

Modèle MLP = modèle séquenciel avec des couches intermédiaires denses (chaque neuronne est connecté à tout les neuronnes des couches avant et après). 

Limitations = 
- tendance à surajuster les données (overfitting à voir au Chap3), en particulier lorsque le réseau est très profond ou que le nombre de données d'entraînement est limité. 
- moins efficace pour traiter des données ayant une structure spatiale ou temporelle, comme les images ou les séquences,

Pourquoi moins efficace pour traiter des données spatiales (Images)? 
- **Absence de prise en compte de la localité spatiale ou temporelle** : Dans un MLP, chaque neurone d'une couche est connecté à tous les neurones de la couche précédente, et il traite l'information de manière globale plutôt que locale. Cela signifie qu'un MLP ne prend pas naturellement en compte la proximité ou l'ordre des pixels dans une image, ou des mots/éléments dans une séquence. Par exemple, dans le cas des images, le MLP ne reconnaîtra pas facilement les patterns ou les textures locales, qui sont cruciaux pour la compréhension visuelle.
- **Complexité et sur-paramétrisation** : Pour traiter des images ou des séquences avec un MLP, chaque pixel ou élément de la séquence doit être connecté à plusieurs neurones, ce qui peut rapidement augmenter le nombre de paramètres du modèle. Cela rend le réseau complexe et susceptible de surajuster, surtout lorsque la quantité de données d'entraînement est limitée.

In [5]:
import tensorflow as tf
network = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(28 * 28,)),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compilation du modèle
network.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

#### Pour info 

Fonction d'activation = un neuronne reçoit la somme de chaque valeur des neuronnes de la couche précédentes multipliée par son poids associé. On ajoute ensuite le biais du neuronne à cette somme. Cette valeur passe ensuite par une fonction qui va la transformer en une autre valeur qui sera ensuite passée au(x) neuronne(s) de la couche suivante. Cette fonction d'activation permet de ne pas avoir un réseau linéaire. 

Loss function = fonction de perte, permet de comparer la prédiction obtenue avec la bonne étiquette. 

Optimizer = Va ajuster les poids en fonction de l'output de la fonction de perte. 

**ps : Il existe plusieurs types de fonctions d'activations, de perte et d'optimiseurs. Il faut savoir choisir la bonne fonction pour le modèle en question.**

#### Étape 4 : Entraînement

In [6]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 1s 2ms/step - loss: 0.2587 - accuracy: 0.9253
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1021 - accuracy: 0.9691
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0673 - accuracy: 0.9797
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0494 - accuracy: 0.9849
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0366 - accuracy: 0.9890


In [8]:
# On observe que la précision s'améliore d'epoch en epoch

In [10]:
val_loss, val_acc = network.evaluate(test_images, test_labels)
print(val_loss, val_acc)

313/313 [==============================] - 0s 566us/step - loss: 0.0699 - accuracy: 0.9800
0.06992277503013611 0.9800000190734863
